In [1]:
class ModelArgs:
    def __init__(self, d, nv, nh, drop, ac_conv, ac_fc, L):
        self.d = d
        self.nv = nv
        self.nh = nh
        self.drop = drop
        self.ac_conv = ac_conv
        self.ac_fc = ac_fc
        self.L = L

In [2]:
import torch
from caser import Caser  # Assuming you have a Caser class defined
from interactions import Interactions  # Assuming you have an Interactions class defined
import numpy as np

# Define the model architecture with the correct number of users and items
num_users = 5848  # Replace with the actual number of users used during training
num_items = 235  # Replace with the actual number of items used during training
model_args = ModelArgs(
    d=512,
    nv=4,
    nh=16,
    drop=0.5,
    ac_conv='relu',
    ac_fc='relu',
    L=5  # Set the value for L
)

model = Caser(num_users, num_items, model_args)

# Load the state dictionary
model.load_state_dict(torch.load('thairobotics_finetuned_model.pth', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()

# Load the test data
test_data = Interactions('datasets/coursera_thairobotics/test.txt')

# Transform interactions to sequences
test_data.to_sequence(sequence_length=5, target_length=1)

def predict_for_user(model, user_id, item_ids):
    with torch.no_grad():
        sequences_np = test_data.test_sequences.sequences[user_id, :]
        sequences_np = np.atleast_2d(sequences_np)

        sequences = torch.from_numpy(sequences_np).long()
        item_ids = torch.from_numpy(item_ids).long()
        user_id = torch.from_numpy(np.array([[user_id]])).long()

        # Reshape inputs
        sequences = sequences.unsqueeze(0)
        user_id = user_id.squeeze(1)

        out = model(sequences, user_id, item_ids, for_pred=True)
    return out.cpu().numpy().flatten()

# Example usage
user_id = 0  # Replace with the actual user ID
item_ids = np.array([109])  # Replace with the actual item IDs
print(item_ids)
predictions = predict_for_user(model, user_id, item_ids)
print(predictions)

Total sequences after filtering: 635
[109]
[-6.6100044]


C:\Users\COMPUTER15\AppData\Local\Temp\ipykernel_5976\2622337046.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('thairobotics_finetune

In [3]:
def get_recommendations(model, user_id, num_recommendations=5):
    # ทำนายคะแนนสำหรับคอร์สเรียนทั้งหมด
    item_ids = np.arange(test_data.num_items)
    predictions = predict_for_user(model, user_id, item_ids)
    
    # เลือกคอร์สเรียนที่มีคะแนนสูงสุด
    top_indices = np.argsort(predictions)[-num_recommendations:][::-1]
    top_scores = predictions[top_indices]
    
    return top_indices, top_scores

# ตัวอย่างการใช้งาน
user_id = 0  # แทนที่ด้วย user ID ที่ต้องการ
num_recommendations = 5  # จำนวนคอร์สเรียนที่ต้องการแนะนำ
recommended_courses, scores = get_recommendations(model, user_id, num_recommendations)
print("Recommended Courses:", recommended_courses)
print("Scores:", scores)

Recommended Courses: [ 0 22  2  1 17]
Scores: [2.8250961 1.7911837 1.5722692 1.2321384 1.0935287]


In [4]:
# ฟังก์ชั่นเพื่ออ่านไฟล์ text.txt และสร้างแผนที่ระหว่าง ID ของคอร์สและชื่อคอร์ส
def load_course_names(file_path):
    course_names = {}
    with open(file_path, 'r', encoding='utf-8') as file:  # ระบุการเข้ารหัสเป็น utf-8
        for line in file:
            parts = line.strip().split(' ', 1)  # แยกด้วยช่องว่างครั้งแรกเท่านั้น
            if len(parts) == 2:
                course_id, course_name = parts
                course_names[int(course_id)] = course_name
    return course_names

# โหลดชื่อคอร์สจากไฟล์ text.txt
course_names = load_course_names(r'datasets\coursera_thairobotics\text.txt')

# ฟังก์ชั่นเพื่อแสดงชื่อคอร์สที่แนะนำ
def display_recommended_courses(recommended_courses, course_names):
    for course_id in recommended_courses:
        print(f"Course ID: {course_id}, Course Name: {course_names.get(course_id, 'Unknown')}")

# ตัวอย่างการใช้งาน
user_id = 202  # แทนที่ด้วย user ID ที่ต้องการ
num_recommendations = 5  # จำนวนคอร์สเรียนที่ต้องการแนะนำ
recommended_courses, scores = get_recommendations(model, user_id, num_recommendations)
print("Recommended Courses:", recommended_courses)
print("Scores:", scores)

# แสดงชื่อคอร์สที่แนะนำ
display_recommended_courses(recommended_courses, course_names)

# ฟังก์ชั่นเพื่อดึงข้อมูลการเรียนของผู้ใช้
def get_user_courses(interactions, user_id):
    user_indices = np.where(interactions.user_ids == user_id)[0]
    user_courses = interactions.item_ids[user_indices]
    return user_courses

# โหลดข้อมูลการเรียนจากไฟล์ interactions
interactions = Interactions('datasets/coursera_thairobotics/test.txt')

user_courses = get_user_courses(interactions, user_id)

# แสดงชื่อคอร์สที่ผู้ใช้ได้เรียน
print(f"User {user_id} has taken the following courses:")
for course_id in user_courses:
    print(f"Course ID: {course_id}, Course Name: {course_names.get(course_id, 'Unknown')}")

Recommended Courses: [ 0 22  2 25  5]
Scores: [2.5309947 2.0901287 1.4136686 1.3145612 1.091479 ]
Course ID: 0, Course Name: Support Vector Machine
Course ID: 22, Course Name: Fundamental Mathematics for Robotics (Ordinary Differential Equations)
Course ID: 2, Course Name: PRE – TEST RAC 2566
Course ID: 25, Course Name: Introduction to Deep Learning for Sequential Data
Course ID: 5, Course Name: Introduction to Augmented Reality
User 202 has taken the following courses:
Course ID: 25, Course Name: Introduction to Deep Learning for Sequential Data


In [5]:
import pandas as pd

# อ่านไฟล์ train.txt
train_data = pd.read_csv('datasets/coursera_thairobotics/train.txt', sep=' ', header=None, names=['user_id', 'course_id', 'interaction'])

# กรองข้อมูลเฉพาะ user_id = 0
user_0_courses = train_data[train_data['user_id'] == 0]

# แสดงคอร์สที่ user 0 เรียน
print("Courses taken by user 0 in the train dataset:")
for _, row in user_0_courses.iterrows():
    course_id = row['course_id']
    course_name = course_names.get(course_id, "Unknown Course")
    print(f"Course ID: {course_id}, Course Name: {course_name}")

# แสดงจำนวนคอร์สที่ user 0 เรียน
print(f"\nTotal number of courses taken by user 0: {len(user_0_courses)}")

Courses taken by user 0 in the train dataset:
Course ID: 8.0, Course Name: Robotic Automation Systems (PLC)
Course ID: 43.0, Course Name: Machine Learning for Robotics
Course ID: 11.0, Course Name: Internet of Things Applications
Course ID: 1.0, Course Name: Dimensionality Reduction
Course ID: 25.0, Course Name: Introduction to Deep Learning for Sequential Data
Course ID: 27.0, Course Name: Introduction to Robotics
Course ID: 24.0, Course Name: Object Localization Using Gradient-Weighted Class Activation Mapping (Grad-CAM)
Course ID: 26.0, Course Name: Introduction to Neural Networks and How it Learns ?

Total number of courses taken by user 0: 8


In [8]:
import random

# แสดง item id ทั้งหมด
print("All item IDs:")
print(list(course_names.keys()))

# สุ่มเลือก 10 ชื่อคอร์ส
random_courses = random.sample(list(course_names.items()), 10)

print("\nRandomly selected 10 courses:")
for course_id, course_name in random_courses:
    print(f"Course ID: {course_id}, Course Name: {course_name}")

All item IDs:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 